In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns


def raw_data():
  data = pd.read_csv('/content/iris.data',names=['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width', 'iris_class'])
  data["iris_class"].replace({"Iris-setosa": 0., "Iris-virginica": 1., "Iris-versicolor": 2.}, inplace=True)
  data1 = data[data.iris_class != 2] # 0 and 1
  data2 = data[data.iris_class != 1] # 0 and 2
  data3 = data[data.iris_class != 0] # 1 and 2
  return data1, data2, data3

def prepared_data():
  data1, data2, data3 = raw_data()
  data2["iris_class"].replace({2:1}, inplace=True)
  data3["iris_class"].replace({1:0, 2:1}, inplace=True)
  X_train1, X_test1, y_train1, y_test1 = train_test_split(data1.iloc[ : , 0:-1], data1.iloc[ : , -1], 
                                                        stratify=data1['iris_class'], test_size=0.2, random_state=2020)
  X_train2, X_test2, y_train2, y_test2 = train_test_split(data2.iloc[ : , 0:-1], data2.iloc[ : , -1],
                                                        stratify=data2['iris_class'], test_size=0.2, random_state=2020)
  X_train3, X_test3, y_train3, y_test3 = train_test_split(data3.iloc[ : , 0:-1], data3.iloc[ : , -1],
                                                      stratify=data3['iris_class'], test_size=0.2, random_state=2020)
  
  X_train1, X_test1 = addbias(X_train1), addbias(X_test1)
  X_train2, X_test2 = addbias(X_train2), addbias(X_test2)
  X_train3, X_test3 = addbias(X_train3), addbias(X_test3)
  res1 = (X_train1, X_test1, y_train1.to_numpy(), y_test1.to_numpy())
  res2 = (X_train2, X_test2, y_train2.to_numpy(), y_test2.to_numpy())
  res3 = (X_train3, X_test3, y_train3.to_numpy(), y_test3.to_numpy())
  return res1, res2, res3

def init_weights(shape):
  return np.zeros(shape, dtype='float64')

def relabel_data(y):
    label = list(set(y))
    relabeled_data = np.zeros(len(y)*len(label)).reshape(len(y),len(label))
    for i in range(len(label)):
        relabeled_data[y==label[i],i] = 1
    return relabeled_data

def addbias(x):
    return np.concatenate((np.ones((len(x))).reshape(-1,1), x),axis = 1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_loss(y_, y):
  return -1/y.size * np.sum(y * np.log(y_) + (1 - y) * np.log(1 - y_), axis=0)

def gradient_dsc(X, y, y_):
  return np.dot(X.T, (y_ - y)) / y.size

def update_weights(w, lr, grad):
  return w - lr * grad.T

def mse(y_, y):
  diff = np.subtract(y_, y)
  ms = np.power(diff, 2, dtype='float64')
  return np.mean(ms)

def train(X_train, y_train, lr, epochs, weights): 
    for i in range(epochs): 
        xw = np.dot(X_train, weights.T)
        prob = sigmoid(xw)
        grad = gradient_dsc(X_train, y_train, prob)
        weights = update_weights(weights, lr, grad)
        loss = compute_loss(prob, y_train)
        loss_log.append(loss)
    return weights

def predict(X, weights):
    z = np.dot(X, weights.T)
    return sigmoid(z)

def predict_label(X, weights):
  labels = []
  for i in range(len(X)):
    temp = [0, 0, 0]
    l1 = 1 if predict(X[i], weights[0]) >= 0.5 else 0
    l2 = 1 if predict(X[i], weights[1]) >= 0.5 else 0
    l3 = 1 if predict(X[i], weights[2]) >= 0.5 else 0
    if l1 == 1:
      temp[1] = temp[1] + 1
    else:
      temp[0] = temp[0] + 1
    if l2 == 1:
      temp[2] = temp[2] + 1
    else:
      temp[0] = temp[0] + 1
    if l3 == 1:
      temp[2] = temp[2] + 1
    else:
      temp[1] = temp[1] + 1
    labels.append(temp)
  return [l.index(max(l)) for l in labels]

loss_log = []
epochs = 20000
lr = 0.01

dataset1, dataset2, dataset3 = prepared_data()
num_of_w = dataset1[0].shape[1]
weights1 = init_weights(num_of_w)
weights2 = init_weights(num_of_w)
weights3 = init_weights(num_of_w)

weights01 = train(dataset1[0], dataset1[2], lr, epochs, weights1)
weights02 = train(dataset2[0], dataset2[2], lr, epochs, weights2)
weights12 = train(dataset3[0], dataset3[2], lr, epochs, weights3)
weights = [weights01, weights02, weights12]

In [ ]:
def prepare_data_for_acc():
  data1, data2, data3 = raw_data()
  X_train1, X_test1, y_train1, y_test1 = train_test_split(data1.iloc[ : , 0:-1], data1.iloc[ : , -1], 
                                                        stratify=data1['iris_class'], test_size=0.2, random_state=2020)
  X_train2, X_test2, y_train2, y_test2 = train_test_split(data2.iloc[ : , 0:-1], data2.iloc[ : , -1],
                                                        stratify=data2['iris_class'], test_size=0.2, random_state=2020)
  X_train3, X_test3, y_train3, y_test3 = train_test_split(data3.iloc[ : , 0:-1], data3.iloc[ : , -1],
                                                      stratify=data3['iris_class'], test_size=0.2, random_state=2020)
  
  X_train1, X_test1 = addbias(X_train1), addbias(X_test1)
  X_train2, X_test2 = addbias(X_train2), addbias(X_test2)
  X_train3, X_test3 = addbias(X_train3), addbias(X_test3)
  res1 = (X_train1, X_test1, y_train1.to_numpy(), y_test1.to_numpy())
  res2 = (X_train2, X_test2, y_train2.to_numpy(), y_test2.to_numpy())
  res3 = (X_train3, X_test3, y_train3.to_numpy(), y_test3.to_numpy())
  return res1, res2, res3

In [ ]:
# report train and test accuracy for both of method
from sklearn.metrics import accuracy_score

dataset1, dataset2, dataset3 = prepare_data_for_acc()
y_pred = predict_label(dataset2[0], weights)
y_true = dataset2[2]
print("Train Accuracy : ", accuracy_score(y_true, y_pred))

y_pred = predict_label(dataset2[1], weights)
y_true = dataset2[3]
print("Test Accuracy : ", accuracy_score(y_true, y_pred))

Train Accuracy :  0.9625
Test Accuracy :  1.0
